In [4]:
import requests
import json
import pandas as pd
from datetime import datetime
from pandas.io.json import json_normalize

In [5]:
def fetchStockData(symbol):
    querystring = {"symbol":symbol}
    
    ##Belinda's
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-financials"
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "b0dc7caab0mshbc1cd88ad2aee7bp10be01jsnd242fc2bbf69"
        }
    
    ##PB's
#     headers = {
#     'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
#     'x-rapidapi-key': "aaf8b66fefmsh0021683065f6bddp18ea51jsn7b4e9b5a44cd"
#     }
    
    response = requests.request("GET", url, headers=headers, params=querystring)
    if(response.ok):
        return response.json()
    else:
        return None

In [6]:
# url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/AAPL/cashflow-statement"

In [7]:
# headers = {
#     'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
#     'x-rapidapi-key': "aaf8b66fefmsh0021683065f6bddp18ea51jsn7b4e9b5a44cd"
#     }

In [8]:
r_json = fetchStockData('AAPL')

In [9]:
balanceSheetStatements = pd.json_normalize(r_json['balanceSheetHistoryQuarterly'], record_path=['balanceSheetStatements'],sep="_")
incomeStatementHistory = pd.json_normalize(r_json['incomeStatementHistoryQuarterly'], record_path=['incomeStatementHistory'],sep="_")
cashflowStatements = pd.json_normalize(r_json['cashflowStatementHistoryQuarterly'], record_path=['cashflowStatements'],sep="_")

In [10]:
raw_cols_bs = [i for i in balanceSheetStatements.columns if 'raw' in i]
balanceSheetStatements = balanceSheetStatements[raw_cols_bs]
raw_cols_is = [i for i in incomeStatementHistory.columns if 'raw' in i]
incomeStatementHistory = incomeStatementHistory[raw_cols_is]
raw_cols_cs = [i for i in cashflowStatements.columns if 'raw' in i]
cashflowStatements = cashflowStatements[raw_cols_cs]

In [11]:
balanceSheetStatements['endDate_raw'] = pd.to_datetime(balanceSheetStatements['endDate_raw'],unit='s')
incomeStatementHistory['endDate_raw'] = pd.to_datetime(incomeStatementHistory['endDate_raw'],unit='s')
cashflowStatements['endDate_raw'] = pd.to_datetime(cashflowStatements['endDate_raw'],unit='s')

In [12]:
balanceSheetStatements = balanceSheetStatements.set_index('endDate_raw')
incomeStatementHistory = incomeStatementHistory.set_index('endDate_raw')
cashflowStatements = cashflowStatements.set_index('endDate_raw')

In [13]:
balanceSheetStatements.loc['Total',:]= balanceSheetStatements.sum(axis=0)
incomeStatementHistory.loc['Total',:]= incomeStatementHistory.sum(axis=0)
cashflowStatements.loc['Total',:]= cashflowStatements.sum(axis=0)

In [14]:
balanceSheetStatements.head()

,totalLiab_raw,totalStockholderEquity_raw,otherCurrentLiab_raw,totalAssets_raw,commonStock_raw,otherCurrentAssets_raw,retainedEarnings_raw,otherLiab_raw,treasuryStock_raw,otherAssets_raw,...,otherStockholderEquity_raw,propertyPlantEquipment_raw,totalCurrentAssets_raw,longTermInvestments_raw,netTangibleAssets_raw,shortTermInvestments_raw,netReceivables_raw,longTermDebt_raw,inventory_raw,accountsPayable_raw
endDate_raw,,,,,,,,,,,,,,,,,,,,,
2020-06-27 00:00:00,2.450620e+11,7.228200e+10,3.994500e+10,3.173440e+11,4.869600e+10,1.098700e+10,2.413600e+10,4.760600e+10,-5.500000e+08,3.283600e+10,...,-5.500000e+08,4.385100e+10,1.400650e+11,1.005920e+11,7.228200e+10,5.964200e+10,3.207500e+10,9.404800e+10,3.978000e+09,3.532500e+10
2020-03-28 00:00:00,2.419750e+11,7.842500e+10,4.204800e+10,3.204000e+11,4.803200e+10,1.569100e+10,3.318200e+10,4.874500e+10,-2.789000e+09,3.386800e+10,...,-2.789000e+09,4.398600e+10,1.437530e+11,9.879300e+10,7.842500e+10,5.387700e+10,3.067700e+10,8.908600e+10,3.334000e+09,3.242100e+10
2019-12-28 00:00:00,2.510870e+11,8.953100e+10,4.057700e+10,3.406180e+11,4.597200e+10,1.202600e+10,4.397700e+10,4.864800e+10,-4.180000e+08,3.319500e+10,...,-4.180000e+08,4.429300e+10,1.632310e+11,9.989900e+10,8.953100e+10,6.739100e+10,3.994600e+10,9.307800e+10,4.097000e+09,4.511100e+10
2019-09-28 00:00:00,2.480280e+11,9.048800e+10,4.324200e+10,3.385160e+11,4.517400e+10,1.235200e+10,4.589800e+10,5.050300e+10,-5.840000e+08,3.297800e+10,...,-5.840000e+08,3.737800e+10,1.628190e+11,1.053410e+11,9.048800e+10,5.171300e+10,4.580400e+10,9.180700e+10,4.106000e+09,4.623600e+10
Total,9.861520e+11,3.307260e+11,1.658120e+11,1.316878e+12,1.878740e+11,5.105600e+10,1.471930e+11,1.955020e+11,-4.341000e+09,1.328770e+11,...,-4.341000e+09,1.695080e+11,6.098680e+11,4.046250e+11,3.307260e+11,2.326230e+11,1.485020e+11,3.680190e+11,1.551500e+10,1.590930e+11
